# Neural network hybrid recommendation system on Google Analytics data preprocessing

This notebook demonstrates how to implement a hybrid recommendation system using a neural network to combine content-based and collaborative filtering recommendation models using Google Analytics data. We are going to use the learned user embeddings from [wals.ipynb](../wals.ipynb) and combine that with our previous content-based features from [content_based_using_neural_networks.ipynb](../content_based_using_neural_networks.ipynb)

First we are going to preprocess our data using BigQuery and Cloud Dataflow to be used in our later neural network hybrid recommendation model.

Apache Beam only works in Python 2 at the moment, so we're going to switch to the Python 2 kernel. In the above menu, click the dropdown arrow and select `python2`.

In [ ]:
%%bash
source activate py2env
pip uninstall -y google-cloud-dataflow
conda install -y pytz==2018.4
pip install apache-beam[gcp]

Now restart notebook's session kernel!

In [2]:
# Import helpful libraries and setup our project, bucket, and region
import os

PROJECT = 'cloud-training-demos' # REPLACE WITH YOUR PROJECT ID
BUCKET = 'cloud-training-demos-ml' # REPLACE WITH YOUR BUCKET NAME
REGION = 'us-central1' # REPLACE WITH YOUR BUCKET REGION e.g. us-central1

# do not change these
os.environ['PROJECT'] = PROJECT
os.environ['BUCKET'] = BUCKET
os.environ['REGION'] = REGION
os.environ['TFVERSION'] = '1.8'

In [ ]:
%bash
gcloud  config  set project $PROJECT
gcloud config set compute/region $REGION

<h2> Create ML dataset using Dataflow </h2>
Let's use Cloud Dataflow to read in the BigQuery data, do some preprocessing, and write it out as CSV files.

First, let's create our hybrid dataset query that we will use in our Cloud Dataflow pipeline. This will combine some content-based features and the user and item embeddings learned from our WALS Matrix Factorization Collaborative filtering lab that we extracted from our trained WALSMatrixFactorization Estimator and uploaded to BigQuery.

In [4]:
query_hybrid_dataset = """
WITH CTE_site_history AS (
  SELECT
      fullVisitorId as visitor_id,
      (SELECT MAX(IF(index = 10, value, NULL)) FROM UNNEST(hits.customDimensions)) AS content_id,
      (SELECT MAX(IF(index = 7, value, NULL)) FROM UNNEST(hits.customDimensions)) AS category, 
      (SELECT MAX(IF(index = 6, value, NULL)) FROM UNNEST(hits.customDimensions)) AS title,
      (SELECT MAX(IF(index = 2, value, NULL)) FROM UNNEST(hits.customDimensions)) AS author_list,
      SPLIT(RPAD((SELECT MAX(IF(index = 4, value, NULL)) FROM UNNEST(hits.customDimensions)), 7), '.') AS year_month_array,
      LEAD(hits.customDimensions, 1) OVER (PARTITION BY fullVisitorId ORDER BY hits.time ASC) AS nextCustomDimensions
  FROM 
    `cloud-training-demos.GA360_test.ga_sessions_sample`,   
     UNNEST(hits) AS hits
   WHERE 
     # only include hits on pages
      hits.type = "PAGE"
      AND
      fullVisitorId IS NOT NULL
      AND
      hits.time != 0
      AND
      hits.time IS NOT NULL
      AND
      (SELECT MAX(IF(index = 10, value, NULL)) FROM UNNEST(hits.customDimensions)) IS NOT NULL
),
CTE_training_dataset AS (
SELECT
  (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(nextCustomDimensions)) AS next_content_id,
  
  visitor_id,
  content_id,
  category,
  REGEXP_REPLACE(title, r",", "") AS title,
  REGEXP_EXTRACT(author_list, r"^[^,]+") AS author,
  DATE_DIFF(DATE(CAST(year_month_array[OFFSET(0)] AS INT64), CAST(year_month_array[OFFSET(1)] AS INT64), 1), DATE(1970, 1, 1), MONTH) AS months_since_epoch
FROM
  CTE_site_history
WHERE (SELECT MAX(IF(index=10, value, NULL)) FROM UNNEST(nextCustomDimensions)) IS NOT NULL)

SELECT
  CAST(next_content_id AS STRING) AS next_content_id,
  
  CAST(training_dataset.visitor_id AS STRING) AS visitor_id,
  CAST(training_dataset.content_id AS STRING) AS content_id,
  CAST(IFNULL(category, 'None') AS STRING) AS category,
  CONCAT("\\"", REPLACE(TRIM(CAST(IFNULL(title, 'None') AS STRING)), "\\"",""), "\\"") AS title,
  CAST(IFNULL(author, 'None') AS STRING) AS author,
  CAST(months_since_epoch AS STRING) AS months_since_epoch,
  
  IFNULL(user_factors._0, 0.0) AS user_factor_0,
  IFNULL(user_factors._1, 0.0) AS user_factor_1,
  IFNULL(user_factors._2, 0.0) AS user_factor_2,
  IFNULL(user_factors._3, 0.0) AS user_factor_3,
  IFNULL(user_factors._4, 0.0) AS user_factor_4,
  IFNULL(user_factors._5, 0.0) AS user_factor_5,
  IFNULL(user_factors._6, 0.0) AS user_factor_6,
  IFNULL(user_factors._7, 0.0) AS user_factor_7,
  IFNULL(user_factors._8, 0.0) AS user_factor_8,
  IFNULL(user_factors._9, 0.0) AS user_factor_9,
  
  IFNULL(item_factors._0, 0.0) AS item_factor_0,
  IFNULL(item_factors._1, 0.0) AS item_factor_1,
  IFNULL(item_factors._2, 0.0) AS item_factor_2,
  IFNULL(item_factors._3, 0.0) AS item_factor_3,
  IFNULL(item_factors._4, 0.0) AS item_factor_4,
  IFNULL(item_factors._5, 0.0) AS item_factor_5,
  IFNULL(item_factors._6, 0.0) AS item_factor_6,
  IFNULL(item_factors._7, 0.0) AS item_factor_7,
  IFNULL(item_factors._8, 0.0) AS item_factor_8,
  IFNULL(item_factors._9, 0.0) AS item_factor_9,
  
  FARM_FINGERPRINT(CONCAT(CAST(visitor_id AS STRING), CAST(content_id AS STRING))) AS hash_id
FROM CTE_training_dataset AS training_dataset
LEFT JOIN `cloud-training-demos.GA360_test.user_factors` AS user_factors
  ON CAST(training_dataset.visitor_id AS FLOAT64) = CAST(user_factors.user_id AS FLOAT64)
LEFT JOIN `cloud-training-demos.GA360_test.item_factors` AS item_factors
  ON CAST(training_dataset.content_id AS STRING) = CAST(item_factors.item_id AS STRING)
"""

Let's pull a sample of our data into a dataframe to see what it looks like.

In [34]:
import google.datalab.bigquery as bq
df_hybrid_dataset = bq.Query(query_hybrid_dataset + "LIMIT 100").execute().result().to_dataframe()
df_hybrid_dataset.head()

,next_content_id,visitor_id,content_id,category,title,author,months_since_epoch,user_factor_0,user_factor_1,user_factor_2,...,item_factor_1,item_factor_2,item_factor_3,item_factor_4,item_factor_5,item_factor_6,item_factor_7,item_factor_8,item_factor_9,hash_id
0,299837992,1009843101664575397,299836255,News,"""Blümel Kneissl &Co.: Das sind die Fixstarter""",None,574,-0.000061,0.000059,-0.000262,...,-5.072439e-05,7.677825e-04,1.595652e-04,3.168983e-04,-4.565390e-04,1.829965e-04,-6.903299e-04,8.621884e-04,1.151190e-04,-4494117762649118802
1,299823332,1004737016910093870,299836255,News,"""Blümel Kneissl &Co.: Das sind die Fixstarter""",None,574,-0.002599,0.006136,0.001282,...,-5.072439e-05,7.677825e-04,1.595652e-04,3.168983e-04,-4.565390e-04,1.829965e-04,-6.903299e-04,8.621884e-04,1.151190e-04,819361722659452154
2,299833840,1004737016910093870,299823332,News,"""Öfter dicke Luft in Graz als die EU erlaubt""",Elisabeth Holzer,574,-0.002599,0.006136,0.001282,...,9.751300e-02,1.523030e-01,-9.457193e-02,1.342956e-03,-1.282159e-01,-1.001351e-01,-1.045776e-01,1.051101e-01,8.547133e-02,4234722509566641726
3,299811137,1005952977992670243,299352779,News,"""Kurier TV-News: Wien bleibt anders""",Yvonne Widler,574,-0.000357,0.000269,0.000133,...,-1.183177e-30,-6.456697e-30,-2.306365e-30,-2.009890e-30,-6.561971e-30,2.930605e-30,4.523217e-30,-2.453961e-30,5.803576e-30,-2848250255722063362
4,299352779,1005952977992670243,299811137,News,"""Nach einem Monat Koalitionsverhandlungen: Die...",Peter Temel,574,-0.000357,0.000269,0.000133,...,-9.746066e-12,-9.574265e-12,-1.785746e-11,-8.968286e-12,-1.140654e-11,8.132423e-13,9.178777e-12,3.883378e-12,4.624732e-12,8265998422844778115


In [5]:
df_hybrid_dataset.describe()

,user_factor_0,user_factor_1,user_factor_2,user_factor_3,user_factor_4,user_factor_5,user_factor_6,user_factor_7,user_factor_8,user_factor_9,...,item_factor_1,item_factor_2,item_factor_3,item_factor_4,item_factor_5,item_factor_6,item_factor_7,item_factor_8,item_factor_9,hash_id
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,1.000000e+02,1.000000e+02,100.000000,1.000000e+02,100.000000,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02
mean,-0.000147,0.000685,-0.000627,-0.001150,-0.000492,-0.000155,0.000444,-0.000552,0.000201,0.000758,...,7.296223e-01,-3.078193e-01,-0.071049,-3.969388e-01,0.496630,-5.494907e-01,-2.779562e-01,4.003812e-01,-1.501421e-01,4.254542e+17
std,0.001493,0.001397,0.001859,0.002484,0.000871,0.003032,0.001697,0.001980,0.001692,0.001777,...,4.559783e+00,3.366933e+00,0.574152,1.860872e+00,2.775310,4.605988e+00,1.442863e+00,3.901654e+00,1.664435e+00,5.206981e+18
min,-0.005072,-0.001792,-0.003466,-0.007429,-0.004008,-0.007020,-0.002884,-0.006178,-0.003074,-0.005600,...,-3.037239e-02,-2.270137e+01,-5.523473,-1.159028e+01,-0.265594,-3.237039e+01,-1.250565e+01,-9.549941e+00,-1.115637e+01,-9.126060e+18
25%,-0.000772,0.000005,-0.001001,-0.001109,-0.001173,-0.000269,-0.000124,-0.002153,-0.000349,-0.000005,...,-4.683084e-24,-5.500714e-03,-0.000120,-2.404522e-03,-0.000457,-4.063126e-07,-3.291267e-03,-1.949149e-04,-2.214674e-23,-3.704620e+18
50%,0.000008,0.000151,-0.000233,0.000031,-0.000637,-0.000249,0.000125,-0.000320,-0.000051,0.000139,...,2.199448e-13,2.015904e-24,0.000000,-1.777117e-17,0.000000,0.000000e+00,-1.460821e-16,-1.102839e-35,1.785944e-15,2.377111e+17
75%,0.000025,0.001103,0.000263,0.000126,0.000096,0.002153,0.001355,-0.000050,0.000011,0.001451,...,4.349357e-02,5.184471e-04,0.000114,8.476977e-17,0.001352,6.587416e-03,9.396294e-21,6.632489e-06,1.329006e-03,5.453835e+18
max,0.005226,0.006763,0.006207,0.000501,0.002388,0.003629,0.007907,0.003185,0.004892,0.005829,...,3.229185e+01,9.632271e+00,0.768243,1.868268e-01,18.775856,5.649576e+00,6.752157e-02,2.671273e+01,5.161455e+00,8.882012e+18


In [ ]:
import apache_beam as beam
import datetime, os

def to_csv(rowdict):
  # Pull columns from BQ and create a line
  import hashlib
  import copy
  CSV_COLUMNS = 'next_content_id,visitor_id,content_id,category,title,author,months_since_epoch'.split(',')
  FACTOR_COLUMNS = ["user_factor_{}".format(i) for i in range(10)] + ["item_factor_{}".format(i) for i in range(10)]
    
  # Write out rows for each input row for each column in rowdict
  data = ','.join(['None' if k not in rowdict else (rowdict[k].encode('utf-8') if rowdict[k] is not None else 'None') for k in CSV_COLUMNS])
  data += ','
  data += ','.join([str(rowdict[k]) if k in rowdict else 'None' for k in FACTOR_COLUMNS])
  yield ('{}'.format(data))
  
def preprocess(in_test_mode):
  import shutil, os, subprocess
  job_name = 'preprocess-hybrid-recommendation-features' + '-' + datetime.datetime.now().strftime('%y%m%d-%H%M%S')

  if in_test_mode:
      print('Launching local job ... hang on')
      OUTPUT_DIR = './preproc/features'
      shutil.rmtree(OUTPUT_DIR, ignore_errors=True)
      os.makedirs(OUTPUT_DIR)
  else:
      print('Launching Dataflow job {} ... hang on'.format(job_name))
      OUTPUT_DIR = 'gs://{0}/hybrid_recommendation/preproc/features/'.format(BUCKET)
      try:
        subprocess.check_call('gsutil -m rm -r {}'.format(OUTPUT_DIR).split())
      except:
        pass

  options = {
      'staging_location': os.path.join(OUTPUT_DIR, 'tmp', 'staging'),
      'temp_location': os.path.join(OUTPUT_DIR, 'tmp'),
      'job_name': job_name,
      'project': PROJECT,
      'teardown_policy': 'TEARDOWN_ALWAYS',
      'no_save_main_session': True
  }
  opts = beam.pipeline.PipelineOptions(flags = [], **options)
  if in_test_mode:
    RUNNER = 'DirectRunner'
  else:
    RUNNER = 'DataflowRunner'
  p = beam.Pipeline(RUNNER, options = opts)
  
  query = query_hybrid_dataset

  if in_test_mode:
    query = query + ' LIMIT 100' 

  for step in ['train', 'eval']:
    if step == 'train':
      selquery = 'SELECT * FROM ({}) WHERE MOD(ABS(hash_id), 10) < 9'.format(query)
    else:
      selquery = 'SELECT * FROM ({}) WHERE MOD(ABS(hash_id), 10) = 9'.format(query)

    (p 
     | '{}_read'.format(step) >> beam.io.Read(beam.io.BigQuerySource(query = selquery, use_standard_sql = True))
     | '{}_csv'.format(step) >> beam.FlatMap(to_csv)
     | '{}_out'.format(step) >> beam.io.Write(beam.io.WriteToText(os.path.join(OUTPUT_DIR, '{}.csv'.format(step))))
    )

  job = p.run()
  if in_test_mode:
    job.wait_until_finish()
    print("Done!")
    
preprocess(in_test_mode = False)

Let's check our files to make sure everything went as expected

In [ ]:
%bash
rm -rf features
mkdir features

In [ ]:
!gsutil -m cp -r gs://{BUCKET}/hybrid_recommendation/preproc/features/*.csv* features/

In [38]:
!head -3 features/*

==> features/eval.csv-00000-of-00001 <==
710535,951784927766849126,710535,News,"Haus aus Marmor und Grabsteinen",None,503,-0.00170100899413,0.00496714003384,0.0040482301265,0.000690933316946,3.52509268851e-05,-0.00172890012618,0.00153049221262,0.00100265210494,0.00228979066014,-0.00201142113656,-5.59943889043e-19,7.42678684608e-19,-1.3985523895e-19,3.42277049416e-19,1.11620765154e-18,2.17990091471e-18,-2.42801472173e-19,1.5953545546e-19,-1.10792405809e-18,-4.38625547901e-19
299818044,6813364694829221327,711895,None,"Impressum KURIER.at",None,553,0.000617411220446,-0.000148811683175,-0.000547810224816,-0.000194783264305,-0.000416739669163,-1.85458611668e-05,-0.000259642780293,-0.000104108628875,-0.000167975216755,0.000182291900273,-6.17342042923,14.5652112961,17.5528583527,3.3229033947,-44.9284629822,29.9998893738,18.7066059113,-14.6920909882,-20.3173618317,-3.7755317688
714241,8640555275627058154,711895,None,"Impressum KURIER.at",None,553,1.48057097249e-05,-1.99350433832e-05,-1.6683281

/usr/local/envs/py2env/lib/python2.7/site-packages/simplejson/encoder.py:296: DeprecationWarning: Interpreting naive datetime as local 2018-10-09 05:16:20.893523. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


<h2> Create vocabularies using Dataflow </h2>

Let's use Cloud Dataflow to read in the BigQuery data, do some preprocessing, and write it out as CSV files.

Now we'll create our vocabulary files for our categorical features.

In [5]:
query_vocabularies = """
SELECT
  CAST((SELECT MAX(IF(index = index_value, value, NULL)) FROM UNNEST(hits.customDimensions)) AS STRING) AS grouped_by
FROM `cloud-training-demos.GA360_test.ga_sessions_sample`,
  UNNEST(hits) AS hits
WHERE
  # only include hits on pages
  hits.type = "PAGE"
  AND (SELECT MAX(IF(index = index_value, value, NULL)) FROM UNNEST(hits.customDimensions)) IS NOT NULL
GROUP BY
  grouped_by
"""

In [ ]:
import apache_beam as beam
import datetime, os

def to_txt(rowdict):
  # Pull columns from BQ and create a line

  # Write out rows for each input row for grouped by column in rowdict
  return '{}'.format(rowdict['grouped_by'].encode('utf-8'))
  
def preprocess(in_test_mode):
  import shutil, os, subprocess
  job_name = 'preprocess-hybrid-recommendation-vocab-lists' + '-' + datetime.datetime.now().strftime('%y%m%d-%H%M%S')

  if in_test_mode:
      print('Launching local job ... hang on')
      OUTPUT_DIR = './preproc/vocabs'
      shutil.rmtree(OUTPUT_DIR, ignore_errors=True)
      os.makedirs(OUTPUT_DIR)
  else:
      print('Launching Dataflow job {} ... hang on'.format(job_name))
      OUTPUT_DIR = 'gs://{0}/hybrid_recommendation/preproc/vocabs/'.format(BUCKET)
      try:
        subprocess.check_call('gsutil -m rm -r {}'.format(OUTPUT_DIR).split())
      except:
        pass

  options = {
      'staging_location': os.path.join(OUTPUT_DIR, 'tmp', 'staging'),
      'temp_location': os.path.join(OUTPUT_DIR, 'tmp'),
      'job_name': job_name,
      'project': PROJECT,
      'teardown_policy': 'TEARDOWN_ALWAYS',
      'no_save_main_session': True
  }
  opts = beam.pipeline.PipelineOptions(flags = [], **options)
  if in_test_mode:
      RUNNER = 'DirectRunner'
  else:
      RUNNER = 'DataflowRunner'
      
  p = beam.Pipeline(RUNNER, options = opts)
  
  def vocab_list(index, name):
    query = query_vocabularies.replace("index_value", "{}".format(index))

    (p 
     | '{}_read'.format(name) >> beam.io.Read(beam.io.BigQuerySource(query = query, use_standard_sql = True))
     | '{}_txt'.format(name) >> beam.Map(to_txt)
     | '{}_out'.format(name) >> beam.io.Write(beam.io.WriteToText(os.path.join(OUTPUT_DIR, '{0}_vocab.txt'.format(name))))
    )

  # Call vocab_list function for each
  vocab_list(10, 'content_id') # content_id
  vocab_list(7, 'category') # category
  vocab_list(2, 'author') # author
  
  job = p.run()
  if in_test_mode:
    job.wait_until_finish()
    print("Done!")
    
preprocess(in_test_mode = False)

Also get vocab counts from the length of the vocabularies

In [ ]:
import apache_beam as beam
import datetime, os

def count_to_txt(rowdict):
  # Pull columns from BQ and create a line

  # Write out count
  return '{}'.format(rowdict['count_number'])
  
def mean_to_txt(rowdict):
  # Pull columns from BQ and create a line

  # Write out mean
  return '{}'.format(rowdict['mean_value'])
  
def preprocess(in_test_mode):
  import shutil, os, subprocess
  job_name = 'preprocess-hybrid-recommendation-vocab-counts' + '-' + datetime.datetime.now().strftime('%y%m%d-%H%M%S')

  if in_test_mode:
      print('Launching local job ... hang on')
      OUTPUT_DIR = './preproc/vocab_counts'
      shutil.rmtree(OUTPUT_DIR, ignore_errors=True)
      os.makedirs(OUTPUT_DIR)
  else:
      print('Launching Dataflow job {} ... hang on'.format(job_name))
      OUTPUT_DIR = 'gs://{0}/hybrid_recommendation/preproc/vocab_counts/'.format(BUCKET)
      try:
        subprocess.check_call('gsutil -m rm -r {}'.format(OUTPUT_DIR).split())
      except:
        pass

  options = {
      'staging_location': os.path.join(OUTPUT_DIR, 'tmp', 'staging'),
      'temp_location': os.path.join(OUTPUT_DIR, 'tmp'),
      'job_name': job_name,
      'project': PROJECT,
      'teardown_policy': 'TEARDOWN_ALWAYS',
      'no_save_main_session': True
  }
  opts = beam.pipeline.PipelineOptions(flags = [], **options)
  if in_test_mode:
      RUNNER = 'DirectRunner'
  else:
      RUNNER = 'DataflowRunner'
      
  p = beam.Pipeline(RUNNER, options = opts)
  
  def vocab_count(index, column_name):
    query = """
SELECT
  COUNT(*) AS count_number
FROM ({})
""".format(query_vocabularies.replace("index_value", "{}".format(index)))

    (p 
     | '{}_read'.format(column_name) >> beam.io.Read(beam.io.BigQuerySource(query = query, use_standard_sql = True))
     | '{}_txt'.format(column_name) >> beam.Map(count_to_txt)
     | '{}_out'.format(column_name) >> beam.io.Write(beam.io.WriteToText(os.path.join(OUTPUT_DIR, '{0}_vocab_count.txt'.format(column_name))))
    )
    
  def global_column_mean(column_name):
    query = """
SELECT
  AVG(CAST({1} AS FLOAT64)) AS mean_value
FROM ({0})
""".format(query_hybrid_dataset, column_name)
    
    (p 
     | '{}_read'.format(column_name) >> beam.io.Read(beam.io.BigQuerySource(query = query, use_standard_sql = True))
     | '{}_txt'.format(column_name) >> beam.Map(mean_to_txt)
     | '{}_out'.format(column_name) >> beam.io.Write(beam.io.WriteToText(os.path.join(OUTPUT_DIR, '{0}_mean.txt'.format(column_name))))
    )
    
  # Call vocab_count function for each column we want the vocabulary count for
  vocab_count(10, 'content_id') # content_id
  vocab_count(7, 'category') # category
  vocab_count(2, 'author') # author
  
  # Call global_column_mean function for each column we want the mean for
  global_column_mean('months_since_epoch') # months_since_epoch
  
  job = p.run()
  if in_test_mode:
    job.wait_until_finish()
    print("Done!")
    
preprocess(in_test_mode = False)

Let's check our files to make sure everything went as expected

In [ ]:
%bash
rm -rf vocabs
mkdir vocabs

In [ ]:
!gsutil -m cp -r gs://{BUCKET}/hybrid_recommendation/preproc/vocabs/*.txt* vocabs/

In [1]:
!head -3 vocabs/*

==> vocabs/author_vocab.txt-00000-of-00001 <==
Wolfgang Atzenhofer
Stefan Hofer
Bernhard Gaul, Christian Böhmer

==> vocabs/category_vocab.txt-00000-of-00001 <==
News
Lifestyle
Stars & Kultur

==> vocabs/content_id_vocab.txt-00000-of-00001 <==
299792293
299965853
299800661


In [ ]:
%bash
rm -rf vocab_counts
mkdir vocab_counts

In [ ]:
!gsutil -m cp -r gs://{BUCKET}/hybrid_recommendation/preproc/vocab_counts/*.txt* vocab_counts/

In [70]:
!head -3 vocab_counts/*

==> vocab_counts/author_vocab_count.txt-00000-of-00001 <==
1103

==> vocab_counts/category_vocab_count.txt-00000-of-00001 <==
3

==> vocab_counts/content_id_vocab_count.txt-00000-of-00001 <==
15634


/usr/local/envs/py2env/lib/python2.7/site-packages/simplejson/encoder.py:296: DeprecationWarning: Interpreting naive datetime as local 2018-10-08 04:49:28.672721. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)
